In [1]:
!pip install pyheif

In [2]:
!pip install catboost

In [3]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import pyheif 
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import numpy as np

import cv2
import random

In [4]:
test_img_names = set(os.listdir('../input/adfasas/test_dataset_test_yl/test'))
train_img_names = set(os.listdir('../input/ddfsfer/train_dataset_train_yl/train'))

In [5]:
test_img_names.intersection(train_img_names)

In [6]:
train_labels_df = pd.read_csv('../input/ddfsfer/train_dataset_train_yl/train.csv', sep=';', index_col=None)
train_labels_df

In [7]:
train_labels_names = set(train_labels_df['image_name'].values)

In [8]:
train_labels_names.intersection(test_img_names)

In [9]:
len(train_labels_names.intersection(train_img_names)) == len(train_img_names)

In [10]:
train_labels_df['image_name'].value_counts().head(40)

# Load models

In [11]:
!git clone https://github.com/ultralytics/yolov5  # clone
# %cd yolov5
!pip install -r yolov5/requirements.txt

### Logo model

In [28]:
import torch
from PIL import Image

CKPT_PATH = "../input/car-models/model_logo_15epoch.pt"
yolov5_logo = torch.hub.load("./yolov5","custom",path=CKPT_PATH,source="local",force_reload=True)

### Car model

In [64]:
# import torch

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, pretrained=True)
# model.classes = [0, 2]  # person and car

import torch
from PIL import Image

CKPT_PATH = "../input/carmodelupdate/new_car_model_up.pt"
yolov5_car = torch.hub.load("./yolov5","custom",path=CKPT_PATH,source="local",force_reload=True)

### Plate model

In [12]:
import torch
from PIL import Image

CKPT_PATH = "../input/newmodel/plate.pt"
yolov5 = torch.hub.load("./yolov5","custom",path=CKPT_PATH,source="local",force_reload=True)

# Logo

In [29]:
train_data_logo = []

for img_name in tqdm(train_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
  results = yolov5_logo(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].cpu().numpy().tolist()
    train_data_logo.append(results)

In [30]:
len(train_data_logo)

In [31]:
train_data_df_logo = pd.DataFrame(train_data_logo, columns = ['image_name', 'x_min_logo', 'y_min_logo', 'x_max_logo', 'y_max_logo', 'conf_logo', 'class_logo'])
train_data_df_logo

In [32]:
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 3))
img = read_image('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df_logo['image_name'][43])
# print('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df['image_name'][0])
bbox = [train_data_df_logo['x_min_logo'][43], train_data_df_logo['y_min_logo'][43],train_data_df_logo['x_max_logo'][43],train_data_df_logo['y_max_logo'][43]]
print(bbox)
bbox = torch.tensor(bbox, dtype=torch.int)
bbox = bbox.unsqueeze(0)
img=draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))

img = torchvision.transforms.ToPILImage()(img)
img

# Plate

In [13]:
train_data_plate = []

for img_name in tqdm(train_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
  results = yolov5(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].cpu().numpy().tolist()
    train_data_plate.append(results)

In [49]:
len(train_data_plate)

In [50]:
train_data_df_plate = pd.DataFrame(train_data_plate, columns = ['image_name', 'x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate', 'class_plate'])
train_data_df_plate

In [51]:
train_data_df_plate.conf_plate.mean()

In [19]:
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 3))
img = read_image('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df_plate['image_name'][3])
# print('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df['image_name'][0])
bbox = [train_data_df_plate['x_min_plate'][3], train_data_df_plate['y_min_plate'][3],train_data_df_plate['x_max_plate'][3],train_data_df_plate['y_max_plate'][3]]
print(bbox)
bbox = torch.tensor(bbox, dtype=torch.int)
bbox = bbox.unsqueeze(0)
img=draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))

img = torchvision.transforms.ToPILImage()(img)
img
# plt.show()

In [62]:
# data_plate_train = pd.read_csv("../input/train-plate-annot/annotation_car_plate_train.csv")
# data_plate_train.rename(columns={'xmin': 'x_min_plate', 'ymin': 'y_min_plate','xmax': 'x_max_plate','ymax': 'y_max_plate','file': 'image_name','Xcent': 'Xcent_plate','Ycent': 'Ycent_plate','boxW': 'boxW_plate','boxH': 'boxH_plate'}, inplace=True)
# data_plate_train

In [63]:
# for i in range(len(data_plate_train.image_name)):
#     data_plate_train["image_name"][i] = data_plate_train["image_name"][i]+'.jpg'
# data_plate_train['class'] = 0
# data_plate_train = data_plate_train.drop("name", axis=1)
# data_plate_train

# Car

In [65]:
train_data = []

for img_name in tqdm(train_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/ddfsfer/train_dataset_train_yl/train/', img_name))
  results = yolov5_car(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].cpu().numpy().tolist()
    train_data.append(results)

In [66]:
train_data_df = pd.DataFrame(train_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class'])
train_data_df

In [25]:
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 3))
i=12
img = read_image('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df['image_name'][i])
# print('../input/ddfsfer/train_dataset_train_yl/train/'+train_data_df['image_name'][0])
bbox = [train_data_df['x_min'][i], train_data_df['y_min'][i],train_data_df['x_max'][i],train_data_df['y_max'][i]]
print(bbox)
bbox = torch.tensor(bbox, dtype=torch.int)
bbox = bbox.unsqueeze(0)
img=draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))

img = torchvision.transforms.ToPILImage()(img)
img

In [53]:
train_data_df.conf.mean()

In [67]:
train_data_df = pd.merge(train_labels_df, train_data_df, how='left')
train_data_df

In [68]:
train_data_df = pd.merge(train_data_df, train_data_df_plate, how='left')
train_data_df

In [42]:
# train_data_df = pd.merge(train_data_df, train_data_df_plate, how='left')
# train_data_df

In [36]:
train_data_df = pd.merge(train_data_df, train_data_df_logo, how='left')
train_data_df

# Features

In [23]:
width = 4032
height = 3024
train_data_df['Xcent']=(train_data_df['x_min']+train_data_df['x_max'])/(2*width)
train_data_df['Ycent']=(train_data_df['y_min']+train_data_df['y_max'])/(2*height)
train_data_df['boxW']=(train_data_df['x_max']-train_data_df['x_min'])/width
train_data_df['boxH']=(train_data_df['y_max']-train_data_df['y_min'])/height

train_data_df['Xcent_plate']=(train_data_df['x_min_plate']+train_data_df['x_max_plate'])/(2*width)
train_data_df['Ycent_plate']=(train_data_df['y_min_plate']+train_data_df['y_max_plate'])/(2*height)
train_data_df['boxW_plate']=(train_data_df['x_max_plate']-train_data_df['x_min_plate'])/width
train_data_df['boxH_plate']=(train_data_df['y_max_plate']-train_data_df['y_min_plate'])/height

# train_data_df['Xcent_logo']=(train_data_df['x_min_logo']+train_data_df['x_max_logo'])/(2*width)
# train_data_df['Ycent_logo']=(train_data_df['y_min_logo']+train_data_df['y_max_logo'])/(2*height)
# train_data_df['boxW_logo']=(train_data_df['x_max_logo']-train_data_df['x_min_logo'])/width
# train_data_df['boxH_logo']=(train_data_df['y_max_logo']-train_data_df['y_min_logo'])/height
train_data_df

In [69]:
(train_data_df[train_data_df.columns[7]].value_counts())

In [70]:
y = train_data_df[['distance']].values
# X = train_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate','conf_plate']]
X = train_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate','Xcent','Ycent','boxW','boxH','Xcent_plate','Ycent_plate','boxW_plate','boxH_plate','conf_plate']]
# X = train_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate','Xcent','Ycent','boxW','boxH','Xcent_plate','Ycent_plate','boxW_plate','boxH_plate','x_min_logo', 'y_min_logo', 'x_max_logo', 'y_max_logo', 'conf_logo','Xcent_logo','Ycent_logo','boxW_logo','boxH_logo']]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1, shuffle=True)

In [72]:
X_train

In [73]:
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

In [56]:
model_2 = xgb.XGBRegressor(seed=123445, learning_rate=0.1,max_depth=4,n_estimators=2000,max_delta_step=5,base_score=0.5, booster='gbtree',reg_alpha=0, reg_lambda=1,verbosity=1,min_child_weight=1) 
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
print(r2_score(y_test, pred))

In [74]:
model_2 = CatBoostRegressor(iterations=5000,depth=4, task_type='CPU',learning_rate=0.01,l2_leaf_reg=2.5, random_seed=45642, loss_function='MAE',logging_level = "Silent",bagging_temperature=0,random_strength=1,eval_metric="R2")
# model_2 = CatBoostRegressor(logging_level = "Silent")
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
print(r2_score(y_test, pred))

In [55]:
model_2 = lgb.LGBMRegressor(learning_rate = 0.01, max_depth=1,num_iterations=1000,num_leaves=128,objective='regression', task = 'train',bagging_freq=10,bagging_fraction=0.7) 
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
print(r2_score(y_test, pred))

# Test Data

In [75]:
test_data = []

for img_name in tqdm(test_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
  results = yolov5_car(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].numpy().tolist()
    test_data.append(results)

In [34]:
test_data_plate = []

for img_name in tqdm(test_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
  results = yolov5(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].numpy().tolist()
    test_data_plate.append(results)

In [53]:
test_data_logo = []

for img_name in tqdm(test_img_names): 
  if 'heic' in img_name:
    heif_file = pyheif.read(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
    img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
  else:
    img = Image.open(os.path.join('../input/adfasas/test_dataset_test_yl/test/', img_name))
  results = yolov5_logo(np.array(img))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    results = [img_name] + results.xyxy[0][0].numpy().tolist()
    test_data_logo.append(results)

In [76]:
test_data_df = pd.DataFrame(test_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class'])
test_data_df

In [77]:
test_data_df

In [78]:
test_data_df_plate = pd.DataFrame(test_data_plate, columns = ['image_name', 'x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate', 'class_plate'])
test_data_df_plate

In [58]:
test_data_df_logo = pd.DataFrame(test_data_logo, columns = ['image_name', 'x_min_logo', 'y_min_logo', 'x_max_logo', 'y_max_logo', 'conf_logo', 'class_logo'])
test_data_df_logo

In [79]:
test_data_df = pd.merge(test_data_df, test_data_df_plate, how='left')
test_data_df

In [60]:
test_data_df = pd.merge(test_data_df, test_data_df_logo, how='left')
test_data_df

In [41]:
width = 4032
height = 3024
test_data_df['Xcent']=(test_data_df['x_min']+test_data_df['x_max'])/(2*width)
test_data_df['Ycent']=(test_data_df['y_min']+test_data_df['y_max'])/(2*height)
test_data_df['boxW']=(test_data_df['x_max']-test_data_df['x_min'])/width
test_data_df['boxH']=(test_data_df['y_max']-test_data_df['y_min'])/height

test_data_df['Xcent_plate']=(test_data_df['x_min_plate']+test_data_df['x_max_plate'])/(2*width)
test_data_df['Ycent_plate']=(test_data_df['y_min_plate']+test_data_df['y_max_plate'])/(2*height)
test_data_df['boxW_plate']=(test_data_df['x_max_plate']-test_data_df['x_min_plate'])/width
test_data_df['boxH_plate']=(test_data_df['y_max_plate']-test_data_df['y_min_plate'])/height

# test_data_df['Xcent_logo']=(test_data_df['x_min_logo']+test_data_df['x_max_logo'])/(2*width)
# test_data_df['Ycent_logo']=(test_data_df['y_min_logo']+test_data_df['y_max_logo'])/(2*height)
# test_data_df['boxW_logo']=(test_data_df['x_max_logo']-test_data_df['x_min_logo'])/width
# test_data_df['boxH_logo']=(test_data_df['y_max_logo']-test_data_df['y_min_logo'])/height
test_data_df

In [80]:
preds = model_2.predict(test_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate','Xcent','Ycent','boxW','boxH','Xcent_plate','Ycent_plate','boxW_plate','boxH_plate']])
# preds = model_2.predict(test_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate']])
# preds = model_2.predict(test_data_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf','x_min_plate', 'y_min_plate', 'x_max_plate', 'y_max_plate', 'conf_plate','Xcent','Ycent','boxW','boxH','Xcent_plate','Ycent_plate','boxW_plate','boxH_plate','x_min_logo', 'y_min_logo', 'x_max_logo', 'y_max_logo', 'conf_logo','Xcent_logo','Ycent_logo','boxW_logo','boxH_logo']])

In [81]:
test_data_df['distance'] = preds
# test_data_df.distance=test_data_df.distance.round(2)
test_data_df

In [82]:
sample_solution_df = test_data_df[['image_name', 'distance']]
sample_solution_df

In [83]:
lost_test_items = []

for file_name in test_img_names - set(sample_solution_df['image_name'].values):
  lost_test_items.append([file_name, 0])

In [84]:
lost_test_items_df = pd.DataFrame(lost_test_items, columns=['image_name', 'distance'])

In [85]:
sample_solution_df = pd.concat([sample_solution_df, lost_test_items_df])
sample_solution_df

In [86]:
sample_solution_df.to_csv('sample_solution44.csv', sep=';', index=False)